<a href="https://colab.research.google.com/github/kyotoman-koshida/bokete/blob/main/notebook/20220930.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まずはResNet152で転移学習を行う

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!pip install transformers[ja]
!pip install --quiet sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.special import softmax

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

from transformers import (
    AutoTokenizer, AutoModel, MMBTForClassification, MMBTConfig, AutoConfig,
    Trainer, TrainingArguments,
)
import transformers

from torchvision.io import read_image
from torchvision.models import ResNet152_Weights, resnet152
from torchvision.models import MobileNet_V2_Weights, mobilenet_v2

from matplotlib import pyplot as plt
import seaborn as sns

In [27]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(42)

画像整形用クラス

In [28]:
class CFG:
    img_size = 224
    batch_size = 17

画像整形用関数

In [29]:
INPUT = "/content/drive/MyDrive/会社/Nishika/bokete" # 所望のディレクトリに変更してください。
train_image_path = "/content/drive/MyDrive/会社/Nishika/bokete/content/train/"
test_image_path = "/content/drive/MyDrive/会社/Nishika/bokete/content/test/"

train_df = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT, "test.csv"))
submission_df = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

# train_df["img_path"] = train_image_path + train_df["odai_photo_file_name"]
train_df["img_path"] = "/content/drive/MyDrive/会社/Nishika/bokete/content/train/" + train_df["odai_photo_file_name"]
# test_df["img_path"] = test_image_path + test_df["odai_photo_file_name"]
test_df["img_path"] = "/content/drive/MyDrive/会社/Nishika/bokete/content/test/" + test_df["odai_photo_file_name"]

# from sklearn.model_selection import train_test_split
# train_df, _ = train_test_split(train_df, train_size=0.7)

In [30]:
import cv2

def resize_to_square(im):
    old_size = im.shape[:2] 
    ratio = float(CFG.img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # 画像サイズを224×224に変更します
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = CFG.img_size - new_size[1]
    delta_h = CFG.img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im


def load_image(ids, is_train=True):
  if is_train:
    # image = cv2.imread(train_image_path+ids)
    image = read_image(train_image_path+ids)
  else:
    # image = cv2.imread(test_image_path+ids)
    image = read_image(test_image_path+ids)
  # new_image = resize_to_square(image)
  # new_image = preprocess_input(new_image)
  new_image = image_transforms(image)#ResNet152用に画像変換
  new_image = new_image.permute((1,2,0))#後のテンソルの規格に合うように列の入れ替え
  return new_image

学習済みResNet152を取得する

In [31]:
pretrained_weight = ResNet152_Weights.IMAGENET1K_V2
model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)
modules = list(model.children())
model = nn.Sequential(*modules)
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [32]:
# 勾配の計算をしないようにする
for param in model.parameters():
    param.requires_grad = False

In [33]:
# モデルのLinear層を変更する
model[-1] = torch.nn.Linear(in_features=2048, out_features=1000, bias=True)# 最後の層だけ訓練し直すから書き換える

In [34]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


訓練用に損失関数とオプティマイザを用意する

In [35]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

画像データのバッチ化

In [36]:
image_df_train = train_df[["id", "odai_photo_file_name"]].copy()
image_df_train.head()

,id,odai_photo_file_name
0,ge5kssftl,9fkys1gb2r.jpg
1,r7sm6tvkj,c6ag0m1lak.jpg
2,yp5aze0bh,whtn6gb9ww.jpg
3,ujaixzo56,6yk5cwmrsy.jpg
4,7vkeveptl,0i9gsa2jsm.jpg


In [37]:
image_ids = image_df_train["odai_photo_file_name"].values
n_batches = len(image_ids) // CFG.batch_size + 1

In [38]:
image_ids

array(['9fkys1gb2r.jpg', 'c6ag0m1lak.jpg', 'whtn6gb9ww.jpg', ...,
       'ks04y4iy7i.jpg', 'cgfkktchbz.jpg', 'jwulli1cmi.jpg'], dtype=object)

In [39]:
n_batches

1469

In [73]:
import pickle
import gc
import math


#ResNet152に突っ込むことができるような画像に整形するためのモジュール
image_transform=ResNet152_Weights.IMAGENET1K_V2
image_transforms = image_transform.transforms()

### 画像の読み込みと整形
tolerance = 300
path_list = list(train_df['img_path'])
is_tensor_empty = True
count = 1
# for i in range(len(path_list)):
#     if i == 0 or (i + 1) % len(path_list) == 0 or (i + 1) % tolerance == 0:
#         print('\rprocessing... jpg num:%s/%s  save point num:%s/%s'%(i + 1, len(path_list), count, math.ceil(len(path_list) / tolerance)))
#         # time.sleep(1)
#     image_tensor = read_image(path_list[i])
    
#     if image_tensor.shape[0] == 1:
#         # 1channel=白黒画像があるので3channelにconvertしています。
#         image_tensor = image_tensor.expand(3, *image_tensor.shape[1:])
    
#     # image = image_transforms(read_jpg(row["img_path"]))
#     image = image_transforms(image_tensor)
#     count += 1


### 整形した画像のバッチ化
# batch_images_tensors = []
# batch_images = np.zeros((CFG.batch_size, len(batch_ids),CFG.img_size,CFG.img_size,3))
batch_images_tensors = [ [] for i in range(CFG.batch_size) ]
batch_group_size = len(image_ids)/CFG.batch_size
for b in range(CFG.batch_size):
    start = b*batch_group_size
    end = (b+1)*batch_group_size
    batch_ids = image_ids[start:end]
    print(batch_ids)
    # pre_batch_images = np.zeros((len(batch_ids),CFG.img_size,CFG.img_size,3))
    pre_batch_images = [ [] for j in batch_ids ]
    for i,image_id in tqdm(enumerate(batch_ids)):
        try:
          pre_batch_images[i].append( load_image(image_id, True) )
        except:
          print("Error")

        # if i == 0 or (i + 1) % len(path_list) == 0 or (i + 1) % tolerance == 0:
        #     print('\rprocessing... jpg num:%s/%s'%(i + 1, len(path_list)))
        # count += 1

    print('\rprocessing... jpg num:%s/%s'%(b + 1, CFG.batch_size))
    batch_images_tensors[b].append(pre_batch_images)

['9fkys1gb2r.jpg' 'c6ag0m1lak.jpg' 'whtn6gb9ww.jpg' '6yk5cwmrsy.jpg'
 '0i9gsa2jsm.jpg' 'eturmndylg.jpg' 'gkjvhzcshz.jpg' 'njkg3ig2xz.jpg'
 '16twkbqjew.jpg' 'nlhjfsckrn.jpg' 'bu6p2rp3nc.jpg' 'zoyqstj6sh.jpg'
 'xzv1cl4d8a.jpg' 'oilltst3qq.jpg' '0ki8xqhkex.jpg' 'curevm1p3c.jpg'
 'j2hj7aafnw.jpg']


0it [00:00, ?it/s]

processing... jpg num:1/17
['0qrjiuxqpi.jpg' 'wnbrkt0cff.jpg' '1ezbfwhd9y.jpg' 'yisqvvj5id.jpg'
 'fiihtgrdb6.jpg' 'cevcfksdhb.jpg' '5emqak5jkz.jpg' '3smdahpngs.jpg'
 'ozp98nilxp.jpg' 'iz60ak9l8d.jpg' 'lntaj1lvgk.jpg' 'tam9barblp.jpg'
 'basgojgwum.jpg' 'kl8ehevbtm.jpg' 'z4lqwlz0vz.jpg' 'fs2nilheob.jpg'
 '9dolikugzq.jpg']


0it [00:00, ?it/s]

processing... jpg num:2/17
['nythhpehok.jpg' 'xkznexuqku.jpg' 'u8dpdxu4hw.jpg' 'cvn3qnr7e3.jpg'
 's1tz7cbzkz.jpg' 'e56txvnkg7.jpg' 'f34qiom5a3.jpg' '7h47biu7vz.jpg'
 'styzda2uls.jpg' 'dj80cuhyio.jpg' '5c1lvnnqad.jpg' 'wbptahic15.jpg'
 '4lpeb8srdv.jpg' 'a2jjxvroeg.jpg' '1pnlpcrboz.jpg' 'fm4p83ykjq.jpg'
 'arwkltmb2i.jpg']


...

processing... jpg num:15/17
['tdzbeyqepo.jpg' 'xw5zobs5iz.jpg' 'y79y29szwy.jpg' 'omzxdksbkx.jpg'
 '4ohxk2rmky.jpg' 'btmrpciidb.jpg' '6kevkrifc7.jpg' 'nqmkvnezqq.jpg'
 'lenqy8z8pv.jpg' 'cnme4gjkvf.jpg' 'jaukjzkbx4.jpg' 'hwhjvsbxii.jpg'
 '0w8woi9ixm.jpg' '8gsjvcewtk.jpg' 'etrnuftzg7.jpg' 'tnxbg7xnck.jpg'
 'aqc1i1jhnp.jpg']


0it [00:00, ?it/s]

processing... jpg num:16/17
['w9bne3ycal.jpg' 'ngpmfdbzhp.jpg' 'arakrk46au.jpg' '9yqs7tpt4w.jpg'
 'ultf7lnvcc.jpg' 'apntwkqh6b.jpg' 'uewb0mqiqe.jpg' '5ua3mhqthb.jpg'
 'j5qc2hd9ep.jpg' 'ghc87ofvpb.jpg' 'fxdtwcf2ke.jpg' 'e9nuxmsgke.jpg'
 'acxxsrscxo.jpg' '4yip3iubmh.jpg' '6nnh200lxg.jpg' 'qlihbrahkd.jpg'
 'vvwdivp1yr.jpg']


0it [00:00, ?it/s]

Error
processing... jpg num:17/17


In [68]:
np.shape(batch_images_tensors[0][0][0])

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = asarray(a).shape


ValueError: ignored

In [71]:
print(batch_images_tensors[0][0][0][0][0][0])

tensor([-0.7479, -0.7402, -0.4450])


↓これを実行するとセッションがクラッシュするので注意

In [ ]:
# with open("/content/drive/MyDrive/会社/Nishika/bokete/bokete_ResNet152.pkl", 'wb') as f:
#         pickle.dump(batch_images_tensors, f)

Linear層だけの訓練をする

In [19]:
# エポックの数（Linear層だけなのでエポック数は少なめ）
epochs = 2

# 訓練ループ
for epoch in range(epochs):

    # ログ用にロスの値を格納する
    running_loss = 0.0

    # エポック
    for i, (inputs, labels) in enumerate(batch_images_tensors, 0):

        # 勾配(グラディエント)をゼロにする
        optimizer.zero_grad()

        # 予測値
        outputs = model(inputs)

        # 損失関数でロスを計算し、逆伝播する
        loss = criterion(outputs, labels)
        loss.backward()

        # モデルの最適化調整
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # 200回に一回、平均のロスを表示
        if (i+1) % 200 == 0:
            print(f'[{epoch+1}, {i+1:5d}] loss: {running_loss/200:.3f}')
            running_loss = 0.0

print('Finished Training')

ValueError: ignored

訓練結果のモデルの重みを保存しておく

In [ ]:
torch.save(model.state_dict(), 'bokete-ResNet152.pt')

In [ ]:
# features = {}
# for b in tqdm(range(n_batches)):
#     start = b*CFG.batch_size
#     end = (b+1)*CFG.batch_size
#     batch_ids = image_ids[start:end]
#     batch_images = np.zeros((len(batch_ids),CFG.img_size,CFG.img_size,3))
#     for i,image_id in enumerate(batch_ids):
#         try:
#             batch_images[i] = load_image(image_id)
#         except:
#           print("Error")
#     batch_preds = m.predict(batch_images)
#     for i,image_id in enumerate(batch_ids):
#         features[image_id] = batch_preds[i]